<h1><center> MODEL </center></h1>

 <center>
    <h1><center>Microsoft malware classification</center></h1>
    <i>Klasifikacija zlonamjernog softvera</i> 
    <br><br>
    PRIRODOSLOVNO MATEMATIČKI FAKULTET - MATEMATIČKI ODSJEK <br>
    <b>Lipanj 2020.</b> 
    <br><br>
    <i>Stanišić Matea, Škrabo Petra, Terzanović Mateja, Tolja Margarita</i>  
</center>

***

[comment]: <> (Paketi koje je potrebno instalirati prije pokretanja bilježnice: <br>)
[comment]: <> (<code>conda install -c anaconda py-xgboost</code> <br>)
[comment]: <> (<code>conda install -c conda-forge imbalanced-learn</code>)

In [23]:
# packages
import numpy as np
from numpy import mean
from numpy import std
from numpy import loadtxt
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import pyplot
from IPython.core.pylabtools import figsize
from collections import Counter
import random

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import precision_score, log_loss, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold, StratifiedKFold, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight

from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [3]:
# load all train features, and data md5hash ids
md5hash_train = pd.read_csv("https://raw.githubusercontent.com/mateastanisic/lfs/master/md5hash_train.csv")
features_train_with_filled_NaN_without_id = pd.read_csv("https://raw.githubusercontent.com/mateastanisic/lfs/master/features_train_with_filled_NaN_without_id.csv")

# load all test features, and data md5hash ids
md5hash_test = pd.read_csv("https://raw.githubusercontent.com/mateastanisic/lfs/master/md5hash_test.csv")
features_test_with_filled_NaN_without_id = pd.read_csv("https://raw.githubusercontent.com/mateastanisic/lfs/master/features_test_with_filled_NaN_without_id.csv")

# load train data malware classes
malware_classes_train = pd.read_csv("https://raw.githubusercontent.com/mateastanisic/lfs/master/malware_classes_train.csv")

In [4]:
# load features from train and test dataset given using rfecv
selected_features_train = pd.read_csv("https://raw.githubusercontent.com/mateastanisic/lfs/master/selected%20features/rfc/selected_features_rfc_train_cv2_186.csv")
selected_features_test = pd.read_csv("https://raw.githubusercontent.com/mateastanisic/lfs/master/selected%20features/rfc/selected_features_rfc_test_cv2_186.csv")

# load table with feature names and their scores, sorted
features_and_importances = pd.read_csv("https://raw.githubusercontent.com/mateastanisic/lfs/master/selected%20features/rfc/rfecv_cv2_186_feature_selected_with_names_and_importance_scores_sorted.csv")

In [13]:
features_and_importances.head()

,feature_name,feature_importance
0,_allmul,0.025009
1,ent_q_diff_diffs_1_min,0.024926
2,ent_q_diff_diffs_1_median,0.021725
3,GetStringTypeA,0.021289
4,byte_1G_a2,0.019825


In [17]:
# groups of features
one_gram_features = features_train_with_filled_NaN_without_id.iloc[:,:256]
metadata_bytes_features = features_train_with_filled_NaN_without_id.iloc[:,256:258]
entropy_features = features_train_with_filled_NaN_without_id.iloc[:,258:460]
image_features = features_train_with_filled_NaN_without_id.iloc[:,460:512]
string_length_features = features_train_with_filled_NaN_without_id.iloc[:,512:628]
metadata_asm_features = features_train_with_filled_NaN_without_id.iloc[:,628:630]
symbols_features = features_train_with_filled_NaN_without_id.iloc[:,630:637]
opcode_features = features_train_with_filled_NaN_without_id.iloc[:,637:730]
reg_features = features_train_with_filled_NaN_without_id.iloc[:,730:756]
section_features = features_train_with_filled_NaN_without_id.iloc[:,756:780]
dd_features = features_train_with_filled_NaN_without_id.iloc[:,780:804]
api_features = features_train_with_filled_NaN_without_id.iloc[:,804:1598]
key_features = features_train_with_filled_NaN_without_id.iloc[:,1598:1693]

In [20]:
# split data into train and test
X_train, X_test, y_train, y_test = train_test_split(features_train_with_filled_NaN_without_id, malware_classes_train.values.ravel(), test_size=0.3, random_state=40)

<br>

## XGBClassifier

In [24]:
#Load model
filename = 'rfecv_rf_cv5_original_features.sav'
rfecv_rf_cv5_original_features = pickle.load(open(filename, 'rb'))

In [25]:
rfecv_rf_cv5_original_features.get_params()

{'cv': StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
 'estimator__bootstrap': True,
 'estimator__ccp_alpha': 0.0,
 'estimator__class_weight': None,
 'estimator__criterion': 'gini',
 'estimator__max_depth': None,
 'estimator__max_features': 'auto',
 'estimator__max_leaf_nodes': None,
 'estimator__max_samples': None,
 'estimator__min_impurity_decrease': 0.0,
 'estimator__min_impurity_split': None,
 'estimator__min_samples_leaf': 1,
 'estimator__min_samples_split': 2,
 'estimator__min_weight_fraction_leaf': 0.0,
 'estimator__n_estimators': 100,
 'estimator__n_jobs': 4,
 'estimator__oob_score': False,
 'estimator__random_state': 0,
 'estimator__verbose': 0,
 'estimator__warm_start': False,
 'estimator': RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='gini', max_depth=None, max_features='auto',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0, min_impu

In [21]:
XGBClassifier().get_params()

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': None,
 'subsample': 1,
 'verbosity': 1}